In [1]:
# prompt: open google drive

from google.colab import drive
drive.mount('/content/drive')
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
bp="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting/colab"
%cd $bp
# !sudo apt-get install nano
# !echo "PS1='\[\033[01;34m\]\W\[\033[00m\]\$ '" >> ~/.bashrc
# !echo "bp='/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/'" >> ~/.bashrc
# !cd ..
print(os.getcwd())
# !git status
!git status --ignore-submodules=all

/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting/colab
Refresh index: 100% (73/73), done.
On branch Leo-mario
Your branch is up to date with 'origin/Leo-mario'.

Last command done (1 command done):
   pick cdc9422 .
Next command to do (1 remaining command):
   pick cdc9422 .
  (use "git rebase --edit-todo" to view and edit)
You are currently editing a commit while rebasing branch 'Leo-mario' on 'e91afd4'.
  (use "git commit --amend" to amend the current commit)
  (use "git rebase --continue" once you are satisfied with your changes)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   ../.gitignore
	new file:   3d-gs.ipynb
	new file:   cloudflared-linux-amd64.deb
	new file:   splat/.gitignore
	new file:   splat/LICENSE
	new file:   splat/README.md
	new file:   splat/convert.py
	new file:   splat/index.html
	new file:   splat/main.js
	new file:  

In [3]:
!git config --global user.email "mariopariona117@gmail.com"
!git config --global user.name "Mario"

In [4]:
!mkdir -p ~/.ssh
!cp "/content/drive/My Drive/ssh_keys/id_ed25519" ~/.ssh/
!cp "/content/drive/My Drive/ssh_keys/id_ed25519.pub" ~/.ssh/
!chmod 600 ~/.ssh/id_ed25519
!eval "$(ssh-agent -s)" && ssh-add ~/.ssh/id_ed25519

Agent pid 3588
Identity added: /root/.ssh/id_ed25519 (mariopariona117@gmail.com)


In [6]:
bp="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting"
%cd $bp
# !git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

# %cd $bp/gaussian-splatting
!pip install -q $bp/submodules/diff-gaussian-rasterization
!pip install -q $bp/submodules/simple-knn

!git config submodule.diff-gaussian-rasterization.update none
!git config submodule.simple-knn.update none
# !git submodule update --init --recursive
#
# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
# !unzip tandt_db.zip
#
# !python train.py -s tandt/train

# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
# !unzip GaussianViewTest.zip
# !python render.py -m /content/gaussian-splatting/GaussianViewTest/model
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/renders.mp4
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/gt.mp4 -y

/content/drive/MyDrive/Machine Visual Perception/3D-GS/gaussian-splatting
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
# bp="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting"
# %cd $bp
# !wget http://storage.googleapis.com/gresearch/refraw360/360_v2.zip
# !unzip 360_v2.zip

In [15]:
# import kagglehub

# bp="/content/drive/MyDrive/Machine\ Visual\ Perception/3D-GS/gaussian-splatting"

# # Download latest version
# path = kagglehub.dataset_download("olwenyu/nerf-llff")

# print("Path to dataset files:", path)
# !mv $path $bp

In [ ]:
# !wget https://storage.googleapis.com/gresearch/refraw360/360_extra_scenes.zip
# !unzip 360_extra_scenes.zip

In [ ]:
DO NOT RUN THIS, IT'S UNDER db ALREADY
# !(wget https://repo-sam.inria.fr/fungraph/deep-blending/data/DrJohnson_Reconstruction_Inputs_Outputs.zip && unzip DrJohnson_Reconstruction_Inputs_Outputs.zip) & \
# (wget https://repo-sam.inria.fr/fungraph/deep-blending/data/Playroom_Reconstruction_Inputs_Outputs.zip && unzip Playroom_Reconstruction_Inputs_Outputs.zip) &

In [ ]:
%cd $bp
print(os.getcwd())
!git clone https://github.com/antimatter15/splat $bp/splat
import os

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O $base_path/cloudflared-linux-amd64.deb
!dpkg -i $bp/cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd $base_path/splat
!python -m http.server 7860

In [ ]:
# prompt: use splat/convert.py to convert the .ply in gaussian-splatting/output/... and then run this as the previous cell is doing (do the rest of the code), that is the most important thing
%cd $base_path
print(os.getcwd())
import os

# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O $base_path/cloudflared-linux-amd64.deb
# !dpkg -i $base_path/cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
%cd $base_path/splat
output = f"{base_path}/gaussian-splatting/output"
!python convert.py $output/73b9c001-a/point_cloud/iteration_7000/point_cloud.ply

#The rest of your code
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd $base_path/splat
!python -m http.server 7860